<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepNLP-END2.0/blob/main/14_BERT_BART/BART_For_Paraphrasing_w_Simple_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install simpletransformers==0.61.13 --quiet

     |████████████████████████████████| 221 kB 8.9 MB/s 
     |████████████████████████████████| 1.2 MB 60.9 MB/s 
     |████████████████████████████████| 1.6 MB 61.3 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 3.3 MB 55.7 MB/s 
     |████████████████████████████████| 76 kB 6.2 MB/s 
     |████████████████████████████████| 264 kB 74.2 MB/s 
     |████████████████████████████████| 124 kB 66.5 MB/s 
     |████████████████████████████████| 8.0 MB 59.6 MB/s 
     |████████████████████████████████| 2.6 MB 52.8 MB/s 
     |████████████████████████████████| 636 kB 53.6 MB/s 
     |████████████████████████████████| 895 kB 59.9 MB/s 
     |████████████████████████████████| 133 kB 63.3 MB/s 
     |████████████████████████████████| 170 kB 63.4 MB/s 
     |████████████████████████████████| 97 kB 8.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 243 kB 71.6 MB/s 
     |█████████████████

In [ ]:
%%bash

mkdir data
wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P data
tar -xvf data/paws_wiki_labeled_final.tar.gz -C data
mv data/final/* data
rm -r data/final

wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv -P data

In [ ]:
%%file utils.py
import warnings

import pandas as pd


def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

Writing utils.py


In [ ]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

from utils import load_data, clean_unnecessary_spaces


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

2021-08-12 00:03:29.736534: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
# Google Data
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/dev.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

# Quora Data

# The Quora Dataset is not separated into train/test, so we do it manually the first time.
df = load_data(
    "data/quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
)
q_train, q_test = train_test_split(df)

q_train.to_csv("data/quora_train.tsv", sep="\t")
q_test.to_csv("data/quora_test.tsv", sep="\t")

# The code block above only needs to be run once.
# After that, the two lines below are sufficient to load the Quora dataset.

# q_train = pd.read_csv("data/quora_train.tsv", sep="\t")
# q_test = pd.read_csv("data/quora_test.tsv", sep="\t")

train_df = pd.concat([train_df, q_train])
eval_df = pd.concat([eval_df, q_test])

train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
len(train_df), len(eval_df)

(133776, 40855)

In [ ]:
train_df

,prefix,input_text,target_text
1,paraphrase,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...
3,paraphrase,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...
4,paraphrase,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...
5,paraphrase,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro, V..."
7,paraphrase,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...
...,...,...,...
391149,paraphrase,What business opportunities may arise as a con...,What are the business opportunities Brexit wil...
274698,paraphrase,What are the easy ways to earn money online?,What is best way to make money online?
217018,paraphrase,How can we get the best grades in school?,How can I get the best grades at school?
96172,paraphrase,How can one overcome porn addiction?,What is the best way to overcome an porn addic...


In [ ]:
train_df_sample = train_df.sample(frac=0.2, random_state=42)
eval_df_sample = eval_df.sample(frac=0.05, random_state=42)

In [ ]:
len(train_df_sample), len(eval_df_sample)

(26755, 8171)

In [ ]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 16
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 16
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"

In [ ]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
)

In [ ]:
model.train_model(train_df_sample, eval_data=eval_df_sample)

In [ ]:
model.eval_batch_size = 16

In [ ]:
%env TOKENIZERS_PARALLELISM=0
model.eval_model(eval_df_sample)

W&B Training Logs: https://wandb.ai/satyajit_meow/Paraphrasing%20with%20BART?workspace=user-satyajit_meow

In [ ]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="outputs/checkpoint-1673-epoch-1",
    args=model_args,
)

In [ ]:
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df_sample["prefix"].tolist(), eval_df_sample["input_text"].tolist())
]
truth = eval_df_sample["target_text"].tolist()

In [ ]:
preds = model.predict(to_predict)

Generating outputs:   0%|          | 0/128 [00:00<?, ?it/s]

In [ ]:
to_predict[:5]

['paraphrase: How can I be less insecure about my short height?',
 'paraphrase: What are tips for saving money?',
 "paraphrase: Why November 14, birthday of Jawaharlal is celebrated as Children's day? What exceptional thing did he do to be considered as dearest to children?",
 'paraphrase: What is it like to visit India?',
 'paraphrase: Is a third world war coming?']

In [ ]:
preds[:5]

[['How can I be less insecure about my height?',
  'How can I be less insecure about my height?',
  'How can I be less insecure about my height?'],
 ['What are some tips for saving money?',
  'What are some tips for saving money?',
  'What are some tips for saving money?'],
 ["Why is the birthday of Jawaharlal Nehru celebrated as Children's Day? What exceptional thing did he do to be considered as dearest to children?",
  "Why is the birthday of Jawaharlal Nehru celebrated as Children's Day? What exceptional thing did he do to be considered dearest to children?",
  "Why is the birthday of Jawaharlal Nehru celebrated as Children's Day? What exceptional thing did he do to be considered dearest to children?"],
 ['What is it like to visit India?',
  'What is it like to visit India?',
  'What is it like to visit India?'],
 ['Is there a third world war on the way?',
  'Is there a third world war on the way?',
  'Is there a third World War coming?']]

In [ ]:
# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(to_predict):
        f.write(str(text).split(':', 1)[1].strip() + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )

In [ ]:
import random

for i, text in enumerate(to_predict[:128]):
    print(f'Text  > {str(text).split(":", 1)[1].strip()}')
    print(f'Pred  < {preds[i][0]}')
    print(f'Truth = {truth[i]}')
    print()

Text  > How can I be less insecure about my short height?
Pred  < How can I be less insecure about my height?
Truth = How can I be less insecure about my short height and looks?

Text  > What are tips for saving money?
Pred  < What are some tips for saving money?
Truth = What are the best methods of saving money?

Text  > Why November 14, birthday of Jawaharlal is celebrated as Children's day? What exceptional thing did he do to be considered as dearest to children?
Pred  < Why is the birthday of Jawaharlal Nehru celebrated as Children's Day? What exceptional thing did he do to be considered as dearest to children?
Truth = Why we celebrate Nehru's birthday as children's day?

Text  > What is it like to visit India?
Pred  < What is it like to visit India?
Truth = What's it like to visit India?

Text  > Is a third world war coming?
Pred  < Is there a third world war on the way?
Truth = Will there be a World War III soon?

Text  > Can we pursue biotechnology after B.tech in mechanical eng